# Convert Model HF to GGUF

## 1. Git Clone llama.cpp

In [ ]:
!git clone https://github.com/ggml-org/llama.cpp
!cd llama.cpp

## 2. Build llama.cpp

In [ ]:
!cmake -B build
!cmake --build build --config Release

Atau jika llama.cpp digunakan hanya untuk convert GGUF perintahnya seperti dibawah ini:

In [ ]:
!cmake -B build -DLLAMA_CURL=OFF
!cmake --build build --config Release

Dengan perintah diatas maka hanya bisa digunakan untuk convert model ke GGUF saja, menggunakan -DLLAMA_CURL=OFF saat menjalankan cmake  akan mematikan dukungan untuk fitur yang membutuhkan libcurl, seperti:
- llama-download (fitur unduh model dari internet)
- Fitur server llama-server (jika menggunakan fitur external KV cache berbasis remote)
- Fitur chat CLI berbasis RAG dari URL atau API

Namun, **ini tidak akan memengaruhi llama-quantize, karena llama-quantize hanya bekerja pada file .gguf lokal dan tidak bergantung pada libcurl.

## 3. Merged Model

Tujuan Merge Model
1. Membuat model standalone:

   - Saat fine-tune model (misal LoRA atau full fine-tuning), perubahan biasanya hanya tersimpan di file delta atau adapter.
   - Proses merge menyatukan perubahan itu ke dalam satu model lengkap.


2. Distribusi / deployment lebih mudah:

   - Hanya satu file .bin atau .gguf yang berisi seluruh bobot model.
   - Tidak perlu dependensi tambahan seperti LoRA adapter atau base model.


3. Siap untuk kuantisasi dan inference:

   - llama.cpp atau llama-quantize butuh bobot model yang sudah digabung, bukan file delta.
   - Setelah merge, bisa dilakukan kuantisasi ke Q4_K_M, Q8_0, dsb.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

In [ ]:
model_id = "Qwen/Qwen3-4B"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             cache_dir = "model_cache/")

model = PeftModel.from_pretrained(model, "/home/nafis/code/research-llm/NLP/finetuning/FT-qwen3-4b/lora_model")

In [ ]:
model = model.merge_and_unload()

In [ ]:
# save model

model.save_pretrained("merged_model_09092024")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="model_cache/")
tokenizer.save_pretrained("merged_model_09092024")

## 4. convert_hf_to_gguf.py




In [ ]:
!python3 llama.cpp/convert_hf_to_gguf.py /home/nafis/code/research-llm/NLP/finetuning/merged_model_19052025

## 5. llama-quantize

sesuaikan dengan quant_type yang ingin digunakan seperti Q4_K_M

In [ ]:
!/home/nafis/code/research-llm/llama.cpp/build/bin/llama-quantize /home/nafis/code/research-llm/NLP/finetuning/merged_model_19052025/Merged_Model_19052025-4.0B-F16.gguf /home/nafis/code/research-llm/NLP/finetuning/merged_model_19052025/ggml-model-Q4_K_M.gguf Q4_K_M